# Eat Safe, Love

## Notebook Set Up

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [8]:
# assign the uk_food database to a variable name
uk_food = mongo['uk_food']

In [9]:
# review the collections in our database
print(uk_food.list_collection_names())

['establishments']


In [10]:
# assign the collection to a variable
uk = uk_food['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [11]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName':1,'scores.Hygiene':1}
Hygiene = list(uk.find(query,fields))

# Use count_documents to display the number of documents in the result
print(f'There are {uk.count_documents(query)} establishments with a score 20 Hygiene score.')
# Display the first document in the results using pprint
pprint(Hygiene[0])

There are 41 establishments with a score 20 Hygiene score.
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('63dc7a697e7e574922823066'),
 'scores': {'Hygiene': 20}}


In [12]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.json_normalize(Hygiene)
# Display the number of rows in the DataFrame
print(f'There are {hygiene_df.count()[0]} rows.')
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

There are 41 rows.


,_id,BusinessName,scores.Hygiene
0,63dc7a697e7e574922823066,The Chase Rest Home,20
1,63dc7a697e7e5749228233e9,Brenalwood,20
2,63dc7a697e7e5749228236ef,Melrose Hotel,20
3,63dc7a697e7e5749228238e1,Seaford Pizza,20
4,63dc7a697e7e5749228238f1,Golden Palace,20
5,63dc7a697e7e574922824290,Ashby's Butchers,20
6,63dc7a697e7e5749228244b5,South Sea Express Cuisine,20
7,63dc7a697e7e5749228259db,Golden Palace,20
8,63dc7a697e7e574922825e1f,The Tulip Tree,20
9,63dc7a697e7e574922826632,F & S,20


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [13]:
# uk.find_one()

In [14]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
#Rating might be a string
query = {'LocalAuthorityName':{'$regex':'Lond'}, 'RatingValue':{'$nin':['0','1','2','3']}} 
#{'$in':['4','5','6','7','8','9','10','11','12']}}
fields = {'BusinessName':1, 'LocalAuthorityName':1, 'RatingValue':1}
                                                            
results_two = uk.find(query,fields)

# Use count_documents to display the number of documents in the result
print('number of documents is ' + str(uk.count_documents(query)))
print('\n')
# Display the first document in the results using pprint
pprint(results_two[0])

number of documents is 34


{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': '4',
 '_id': ObjectId('63dc7a697e7e574922824a7f')}


In [15]:
# Convert the result to a Pandas DataFrame
df_two = pd.DataFrame(results_two)
# Display the number of rows in the DataFrame
print(f'There are {df_two.count()[0]} rows.')
# Display the first 10 rows of the DataFrame
df_two.head(10)

There are 34 rows.


,_id,BusinessName,RatingValue,LocalAuthorityName
0,63dc7a697e7e574922824a7f,Charlie's,4,City of London Corporation
1,63dc7a697e7e574922824da9,Mv City Cruises Erasmus,5,City of London Corporation
2,63dc7a697e7e5749228258f6,Benfleet Motor Yacht Club,4,City of London Corporation
3,63dc7a697e7e5749228266f0,Tilbury Seafarers Centre,5,City of London Corporation
4,63dc7a697e7e5749228266f5,Coombs Catering t/a The Lock and Key,5,City of London Corporation
5,63dc7a697e7e574922826e00,Mv Valulla,5,City of London Corporation
6,63dc7a6a7e7e574922828f09,Froottree,AwaitingInspection,City of London Corporation
7,63dc7a6a7e7e574922828f0c,Tereza Joanne,5,City of London Corporation
8,63dc7a6a7e7e5749228292d3,Brick Lane Brews,4,City of London Corporation
9,63dc7a6a7e7e5749228292ea,City Bar & Grill,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [16]:
query = {'BusinessName':'Penang Flavours'};
fields = {'geocode.latitude':1,'geocode.longitude':1}
uk.find_one(query,fields)

{'_id': ObjectId('63dc7b3cac4e31d449fd0acd'),
 'geocode': {'longitude': Decimal128('0.08384000'),
  'latitude': Decimal128('51.49014200')}}

In [17]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude_low = 51.49014200 - degree_search
latitude_high = 51.49014200 + degree_search
longitude_low = 0.08384000 - degree_search
longitude_high = 0.08384000 + degree_search

# query = {'$match':{'RatingValue':'5'},'$match'}
# sort =  {'$sort':{'scores.Hygiene':1}}

query = {'RatingValue':'5',\
         'geocode.latitude':{'$gte':latitude_low},\
        'geocode.latitude':{'$lte':latitude_high},\
         'geocode.longitude':{'$gte':longitude_low},\
         'geocode.longitude':{'$lte':longitude_high}\
        }
fields = {'BusinessName':1,'RatingValue':1,'scores.Hygiene':1,'geocode.latitude':1,'geocode.longitude':1}
sort = [('scores.Hygiene',1)]
limit = 5
# Print the results
results_three = list(uk.find(query,fields).sort(sort).limit(limit))

pprint(results_three)

[{'BusinessName': 'Magic Wok',
  'RatingValue': '5',
  '_id': ObjectId('63dc7a697e7e5749228267c7'),
  'geocode': {'latitude': Decimal128('51.196736'),
              'longitude': Decimal128('0.065179')},
  'scores': {'Hygiene': None}},
 {'BusinessName': 'Roochi Ltd',
  'RatingValue': '5',
  '_id': ObjectId('63dc7a697e7e57492282603a'),
  'geocode': {'latitude': Decimal128('51.097521'),
              'longitude': Decimal128('0.034461')},
  'scores': {'Hygiene': None}},
 {'BusinessName': 'Wolfe House Resid. Home',
  'RatingValue': '5',
  '_id': ObjectId('63dc7a697e7e574922827095'),
  'geocode': {'latitude': Decimal128('51.255431'),
              'longitude': Decimal128('0.015813')},
  'scores': {'Hygiene': None}},
 {'BusinessName': 'Sultan Kebab House',
  'RatingValue': '5',
  '_id': ObjectId('63dc7a697e7e574922826c50'),
  'geocode': {'latitude': Decimal128('51.266922'),
              'longitude': Decimal128('0.070589')},
  'scores': {'Hygiene': None}},
 {'BusinessName': 'Northall CPC',
  

In [18]:
# Convert result to Pandas DataFrame
df_three = pd.json_normalize(results_three)
print(f'There are {df_three.count()[0]} rows.')
df_three

There are 5 rows.


,_id,BusinessName,RatingValue,scores.Hygiene,geocode.longitude,geocode.latitude
0,63dc7a697e7e5749228267c7,Magic Wok,5,None,0.065179,51.196736
1,63dc7a697e7e57492282603a,Roochi Ltd,5,None,0.034461,51.097521
2,63dc7a697e7e574922827095,Wolfe House Resid. Home,5,None,0.015813,51.255431
3,63dc7a697e7e574922826c50,Sultan Kebab House,5,None,0.070589,51.266922
4,63dc7a697e7e5749228258c8,Northall CPC,5,None,0.022783,51.006613


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [19]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
match_query = {'$match':{'scores.Hygiene':0}}
group_query = {'$group':{'_id':{'LocalAuthorityName':'$LocalAuthorityName'},'count':{'$sum':1}}}
sort_values = {'$sort':{'count':-1}}
pipeline = [match_query, group_query, sort_values]

# {"$group": {"_id": None, "total": {"$sum": 1}, "details":{"$push":{"groupby": "$_id", "count": "$count"}}}}

results_four = list(uk.aggregate(pipeline))

# Print the number of documents in the result
print(f'There are {len(results_four)} documents in this result\n')
# Print the first 10 results
pprint(results_four[0:10])

There are 55 documents in this result

[{'_id': {'LocalAuthorityName': 'Thanet'}, 'count': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'count': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'count': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'count': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'count': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'count': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'count': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'count': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'count': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'count': 542}]


In [20]:
# Convert the result to a Pandas DataFrame
df_four = pd.json_normalize(results_four)
# Display the number of rows in the DataFrame
print(f'There are {df_four.count()[0]} rows.')
# Display the first 10 rows of the DataFrame
df_four.head(10)

There are 55 rows.


,count,_id.LocalAuthorityName
0,1130,Thanet
1,882,Greenwich
2,713,Maidstone
3,711,Newham
4,686,Swale
5,680,Chelmsford
6,672,Medway
7,607,Bexley
8,586,Southend-On-Sea
9,542,Tendring
